Define some variables and input files

In [ ]:
import os

# data_path = "../../../data/contours/svg"
# svg_file = "shapes.svg"
# svg_file = "transform.svg"

data_path = "."
# svg_file = "ellipse.svg"
svg_file = "mdn_arc_example.svg"


input_file = os.path.join(data_path, svg_file)
# print(os.path.abspath(input_file))
# print(os.listdir(data_path))

output_file = "drawing.mesh"

Load SVG file using `svgpathtools`

In [ ]:
from svgpathtools import Document, Path, Line, QuadraticBezier, CubicBezier, Arc, is_bezier_path, is_bezier_segment, is_path_segment, svg2paths, wsvg

doc = Document(input_file)
#paths = doc.paths_from_group(doc.tree.getroot())
paths = doc.paths()


Some utility functions

In [ ]:
def lerp(a,b,t):
    return (1-t)*a+t*b

def line_to_cubic(line : Line):
    q_0,q_1 = line.bpoints()
    return (CubicBezier(q_0, lerp(q_0, q_1, 1/3), lerp(q_0, q_1, 2/3), q_1), [1,1,1,1])

def quadratic_to_cubic(quad : QuadraticBezier):
    q_0,q_1,q_2 = quad.bpoints()
    return (CubicBezier(q_0, lerp(q_0,q_1, 2/3), lerp(q_1,q_2, 1/3), q_2), [1,1,1,1])

def arc_to_cubic(arc: Arc):
    q_0 = arc.start
    q_3 = arc.end

    # Notes:
    # (1) The shoulder point of the ellipse (point at parameter t=0.5)
    # is at the intersection of the segments from control points 0 to 2 and from 1 to 3; 
    # (2) We have control point positions 0 and 3 and their tangents on the ellipse
    # as well as the midpoint; we need to find control points 1 and 2
    try:
        # Use a scaling factor to extend lines from endpoints to shoulder
        # these lines contain the internal control points c_1 and c_2
        scale_fac = 10

        shoulder = arc.point(.5)
        d_0 = arc.derivative(0)
        d_1 = arc.derivative(1)
        # print(f"""For arc {arc} w/ {d_0=} and {d_1=};\n {arc.theta=}, {arc.phi=}, {arc.rotation=}, {arc.delta=}""")

        # extend the line segment from q3 to shoulder point
        # and find the intersection w/ tangent line @ control point 0
        l_3_1 = Line(q_3, q_3 + scale_fac*(shoulder-q_3))
        l_0_1 = Line(q_0, q_0 + d_0)
        ints_31_01 = l_3_1.intersect(l_0_1)
        # print(f"""Finding intersection @ control point 1\n  {shoulder=}\n  {l_3_1=}\n  {l_0_1=}\n  {ints_31_01=}""")

        # extend the line segment from q0 to shoulder point
        # and find the intersection w/ tangent line @ control point 3
        l_0_2 = Line(q_0, q_0 + scale_fac*(shoulder-q_0))
        l_3_2 = Line(q_3, q_3 - d_1)
        ints_02_32 = l_0_2.intersect(l_3_2)
        # print(f"""Finding intersection @ control point 2\n  {shoulder=}\n  {l_0_2=}\n  {l_3_2=}\n  {ints_02_32=}""")

        c_1 = l_0_1.point(ints_31_01[0][1])
        c_2 = l_3_2.point(ints_02_32[0][1])
        # print(f"""Control points from intersections: CP 1: {c_1}; CP 2 {c_2}""")

        return (CubicBezier(q_0, c_1, c_2, q_3), [3,1,1,3])
    except:
        print(f"""*** Problem with arc {arc}:\n\t {arc.theta=}; {arc.delta=}; {arc.phi=} ***""")

        # as a fall-back, use as_cubic_curves function from svgpathtools
        # which approximates rational curve
        # note, we're currently only taking the first cubic; there might be more.
        for c in arc.as_cubic_curves():
            q_0, q_1 = c.start, c.end
            c_1, c_2 = c.control1, c.control2
            return (CubicBezier(q_0, c_1, c_2, q_1), [3,1,1,3])


def segment_as_cubic(seg):
    if isinstance(seg,Line):
        return line_to_cubic(seg)
    elif isinstance(seg,QuadraticBezier):
        return quadratic_to_cubic(seg)
    elif isinstance(seg, CubicBezier):
        return (seg, [1,1,1,1])
    elif isinstance(seg,Arc):
        return arc_to_cubic(seg)
    else:
        raise Exception(f"'{type(seg)}' type not supported yet")


Print out the paths and segments

In [ ]:
for p_idx, p in enumerate(paths):
    for seg_idx, seg in enumerate(p):
        try:
            
            print(f"""Path: {p=} {seg=} {p.element=} \n{p.transform=}""")
        
            if is_bezier_segment(seg):
                cubic,weights =  segment_as_cubic(seg)
                print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}\n\tas cubic: {cubic}; {weights=}")
            elif isinstance(seg, Arc):
                print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}; theta: {seg.theta}; delta: {seg.delta}; phi: {seg.phi}")

                mid = seg.point(.5)
                print(f"""  [Arc: start: {seg.start}, end {seg.end}, center: {seg.center}, point mid: {mid}""")
                print(f"""  [Arc: c1?: {seg.end + 2*(mid-seg.end)}, c2?: {seg.start + 2*(mid-seg.start)},""")

                cubic,weights =  segment_as_cubic(seg)
                print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}\n\tas cubic: {cubic}; {weights=}")

        except Exception as err:
            print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg=}")
            print(f"parsed unsupported type {type(seg)}. Msg={err}")


Convert paths to mfem mesh of cubic Bezier segments using ASCII output

In [ ]:
import numpy as np

# Create an mfem mesh

header = """
MFEM NURBS mesh v1.0

# MFEM Geometry Types (see fem/geom.hpp):
#
# SEGMENT = 1 | SQUARE = 3 | CUBE = 5
#
# element: <attr> 1 <v0> <v1>
# edge: <idx++> 0 1  <-- idx increases by one each time
# knotvector: <order> <num_ctrl_pts> [knots]; sizeof(knots) is 1+order+num_ctrl_pts
# weights: array of weights corresponding to the NURBS element
# FES: list of control points; vertex control points at top, then interior control points

dimension
1
"""

elem_cnt = 0
vert_cnt = 0

elems = []
edges = []
knots = []

# mfem format lists the endpoints and then the interiors
wgts_ends = []
wgts_ints = []
dof_ends = []
dof_ints = []

print(paths)

for p_idx, p in enumerate(paths):

    if not all(map(is_path_segment, p)):
        continue

    for seg_idx, seg in enumerate(p):

        # if isinstance(seg, Arc):
        #     print(f"""Segment {seg} -- center {seg.center}""")

        cubic, weights = segment_as_cubic(seg)
        # print(f"processing {cubic=} {weights=}")

        elems.append(" ".join(map(str,[p_idx + 1, 1, vert_cnt, vert_cnt + 1])))

        edges.append(f"{elem_cnt} 0 1")

        # Hack -- assume for now that the order is always 3 and weights are always 1
        knots.append("3 4 0 0 0 0 1 1 1 1")
        wgts_ends.append(f"{weights[0]} {weights[3]}")
        wgts_ints.append(f"{weights[1]} {weights[2]}")

        dof_ends.append(" ".join(map(str,[cubic.start.real, cubic.start.imag])))
        dof_ends.append(" ".join(map(str,[cubic.end.real, cubic.end.imag])))
        dof_ints.append(" ".join(map(str,[cubic.control2.real, cubic.control2.imag])))
        dof_ints.append(" ".join(map(str,[cubic.control1.real, cubic.control1.imag])))

        vert_cnt += 2
        elem_cnt += 1 

mfem_file = []
mfem_file.append(header)
mfem_file.append("""
elements
{}
{}
""".format(elem_cnt, "\n".join(elems)))

mfem_file.append("""
boundary
0
""")

mfem_file.append("""
edges
{}
{}
""".format(elem_cnt, "\n".join(edges)))

mfem_file.append(f"""
vertices
{vert_cnt}
""")

mfem_file.append("""
knotvectors
{}
{}
""".format(elem_cnt, "\n".join(knots)))

mfem_file.append("""
weights
{}
{}
""".format("\n".join(wgts_ends), "\n".join(wgts_ints)))

mfem_file.append("""
FiniteElementSpace
FiniteElementCollection: NURBS
VDim: 2
Ordering: 1

{}
{}
""".format("\n".join(dof_ends),"\n".join(dof_ints)))

with open(output_file, mode='w') as f:
    f.write("\n".join(mfem_file))
    print(f"wrote '{output_file}' with {vert_cnt} vertices and {elem_cnt} elements")


In [ ]:
from numpy import pi

arc = Arc(start=175.44022368914975+149.55977631085025j, radius=26+46j, rotation=-45, large_arc=True, sweep=False, end=212.20977631085023+112.79022368914977j)
p_0 = arc.point(0)
p_1 = arc.point(0.00001)
delta = p_1-p_0
d_0 = arc.derivative(0)
print(f"""
      p_0: {p_0}, p_1: {p_1}, 
      delta: {delta}, angle: {np.angle(delta)*180/pi}, mag: {np.abs(delta)}, normalized: {delta/np.abs(delta)}, 
      d_0: {d_0}, angle: {np.angle(d_0)*180/pi}, mag: {np.abs(d_0)}, normalized: {d_0/np.abs(d_0)}""")

l = Line(p_0, p_0 + 10 * delta/np.abs(delta))
print(f"""Control polygon for arc
      <path d="M {l.start.real} {p.start.imag} 
      L {l.end.real} {l.end.imag}"
      stroke="cyan" fill="white" stroke-width="1" fill-opacity="0.1"/>""")


for c in arc.as_cubic_curves():
    c_d_0 = c.control1 - c.start
    print(f"""
          curve: {c}, 
          d_0: {c_d_0}, normed: {c_d_0/np.abs(c_d_0)}""")

    print(f"""Control polygon for arc
      <path d="M {c.start.real} {c.start.imag} 
            L {c.control1.real} {c.control1.imag}
            L {c.control2.real} {c.control2.imag}
            L {c.end.real} {c.end.imag}"
            stroke="orange" fill="white" stroke-width="1" fill-opacity="0.1"/>""")
